In this notebook, I'm going to be collecting the candidate lists of Dutch parties for the 2021 pariamentary elections. In no particular order, just guided by availability, Google hit priority and also driven primarily by the parties in the Peilingwijzer.

# CU

In [2]:
import requests
from lxml import html

import pickle

In [16]:
def load_and_save_page(filename, url):
    try:
        with open(filename, 'rb') as fh:
            page = pickle.load(fh)
    except FileNotFoundError:
        print("Downloading new page")
        page = requests.get(url).content
        with open(filename, 'wb') as fh:
            pickle.dump(page, fh)
    return page

In [10]:
filename = 'candidates_CU_page_20201114.pkl'
url = 'https://www.christenunie.nl/kandidatenlijst'

CU_tree = html.fromstring(load_and_save_page(filename, url))

In [19]:
CU_candidate_divs = CU_tree.xpath('//div[@class="col column2"]')

In [39]:
def remove_punctuation(*strings):
    output = []
    punctuation = "(),.|#"
    for string in strings:
        for char in punctuation:
            string = string.replace(char, '')
        output.append(string)
    return output

In [42]:
CU_candidates = {}
for div in CU_candidate_divs:
    naam = div.xpath('p/strong/text()')[0]
    positie, plaats_jaar, CV = div.xpath('p/text()')
    positie, plaats_jaar = remove_punctuation(positie, plaats_jaar)
    plaats_jaar = plaats_jaar.strip().split()
    jaar = plaats_jaar[-1]
    plaats = " ".join(plaats_jaar[:-1])
    CU_candidates[naam] = {
        'positie': positie.strip(),
        'plaats': plaats,
        "jaar": jaar,
        "CV": CV
    }

# CDA

In [17]:
filename = 'candidates_CDA_page_20201114.pkl'
url = 'https://www.cda.nl/kandidaten-2021'

CDA_tree = html.fromstring(load_and_save_page(filename, url))

In [23]:
CDA_candidate_divs = CDA_tree.xpath('//a[@class="person"]')

In [51]:
CDA_candidates = {}
for div in CDA_candidate_divs:
    naam = div.xpath('h3/span/text()')[0].replace('\t', ' ')
    positie_plaats = div.xpath('h4/span/text()')
    positie_plaats = remove_punctuation(*positie_plaats)[0].split()
    positie = positie_plaats[0]
    plaats = " ".join(positie_plaats[1:])
    CDA_candidates[naam] = {
        'positie': positie.strip(),
        'plaats': plaats,
    }

# D66

In [53]:
import pdfx

In [55]:
filename = "D66.pdf"
url = "https://d66.nl/wp-content/uploads/2020/11/D66_Advieslijst_TK2021.pdf"

In [56]:
pdf = requests.get(url).content
with open(filename, 'wb') as fh:
    pickle.dump(pdf, fh)

In [65]:
pdf = pdfx.PDFx(filename)
text = pdf.get_text().split('\n')

In [120]:
D66_candidates = {}
ix = 0
positie = 1
name_part = None
for line in text[16:]:
    if line == '' or '.' in line or '\x0c' in line:
        continue
    elif line in ['Marijke van ', 'Hind ', 'Marleen ', 'Ojanne ', 'Emma Laurijssens ']:
        name_part = line
    elif ix == 0:
        if name_part is not None:
            naam = name_part + line
            name_part = None
        else:
            naam = line
        D66_candidates[naam] = {'positie': positie}
        positie += 1
        ix += 1
    elif ix == 1:
        D66_candidates[naam]['plaats'] = line.strip()
        ix += 1
    elif ix == 2:
        try:
            D66_candidates[naam]['leeftijd'] = line.split()[0]
        except:
            print(naam)
            print(line)
        ix = 0

# fix order, pdf sucks
for naam, candidate in D66_candidates.items():
    if 6 <= candidate['positie'] < 12:
        candidate['positie'] += 1
    if 23 <= candidate['positie'] < 34:
        candidate['positie'] += 2
    if 29 <= candidate['positie'] <= 32:
        candidate['positie'] += 2
    if 57 <= candidate['positie'] <= 61:
        candidate['positie'] -= 1
D66_candidates['Steven van Weyenberg']['positie'] = 6
D66_candidates['Alexander Hammelburg']['positie'] = 23
D66_candidates['Rens Raemakers']['positie'] = 24
D66_candidates['Sjoerd Warmerdam']['positie'] = 29
D66_candidates['Hind Dekker-Abdulaziz']['positie'] = 30
D66_candidates['Bas de Leeuw']['positie'] = 61